In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f"Train = {train.shape}")
print(f"Test = {test.shape}")

Train = (1460, 81)
Test = (1459, 80)


In [3]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
x_train = train.drop(columns = 'SalePrice')
y_train = train['SalePrice']
x_test = test.copy()
print(f" X_Train shape = {x_train.shape} ")
print(f" Y_Train shape = {y_train.shape} ")
print(f" X_Test shape = {x_test.shape} ")

 X_Train shape = (1460, 80) 
 Y_Train shape = (1460,) 
 X_Test shape = (1459, 80) 


In [5]:
percentages = x_train.isnull().mean()*100
percentages

Id                0.000000
MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      17.739726
LotArea           0.000000
Street            0.000000
Alley            93.767123
LotShape          0.000000
LandContour       0.000000
Utilities         0.000000
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.000000
Exterior2nd       0.000000
MasVnrType        0.547945
MasVnrArea        0.547945
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
                   ...    
HalfBath          0.000000
BedroomAbvGr      0.000000
KitchenAbvGr      0.000000
KitchenQual       0.000000
TotRmsAbvGrd      0.000000
Functional        0.000000
F

In [6]:
num_vars = x_train.select_dtypes(include = ['int64','float64']).keys()

cat_vars = x_train.select_dtypes(include = ['object']).keys()

In [7]:
num_variables = [var for var in num_vars if percentages[var]>0]
num_variables

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [8]:
cat_variables = [var for var in cat_vars if percentages[var]>0]
cat_variables

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [9]:
num_var_mean = ['LotFrontage']
num_var_median = ['MasVnrArea', 'GarageYrBlt']
cat_var_mode = ['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',]
cat_var_missing = [ 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature'] 

In [10]:
# Create pipeline

In [11]:
num_var_mean_imputer = Pipeline(steps = [ ("imputer" , SimpleImputer(strategy='mean')) ] )
num_var_median_imputer = Pipeline(steps= [ ("imputer" , SimpleImputer(strategy='median')) ])
cat_var_mode_imputer = Pipeline(steps = [ ("imputer" , SimpleImputer(strategy='most_frequent')) ] )
cat_var_missing_imputer = Pipeline(steps = [ ("imputer" , SimpleImputer(strategy='constant',fill_value='Missing')) ] )

In [12]:
# create ColumnTransformer

In [13]:
preprocessor = ColumnTransformer( transformers= [('mean_imputer' , num_var_mean_imputer , num_var_mean) , ('median_imputer' , num_var_median_imputer , num_var_median) , ('mode_imputer' , cat_var_mode_imputer , cat_var_mode) , ('missing_imputer' , cat_var_missing_imputer , cat_var_missing)   ] )

In [14]:
preprocessor.fit(x_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('mean_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0))]), ['LotFrontage']), ('median_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=Non...))]), ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])])

In [15]:
preprocessor.named_transformers_['mean_imputer'].named_steps['imputer'].statistics_

array([70.04995837])

In [16]:
train['LotFrontage'].mean()

70.04995836802665

In [17]:
preprocessor.transformers_

[('mean_imputer', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
         verbose=0))]), ['LotFrontage']),
 ('median_imputer', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
         strategy='median', verbose=0))]), ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
         strategy='most_frequent', verbose=0))]), ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('missing_imputer', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value='Missing', missing_values=nan,
         strategy='constant', verbose=0))]), ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'M

In [61]:
x_train_clean = preprocessor.transform(x_train)
x_test_clean = preprocessor.transform(x_test)

In [62]:
x_test_clean

array([[80.0, 0.0, 1961.0, ..., 'Missing', 'MnPrv', 'Missing'],
       [81.0, 108.0, 1958.0, ..., 'Missing', 'Missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'Missing', 'MnPrv', 'Missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'Missing', 'Missing', 'Missing'],
       [62.0, 0.0, 1980.0, ..., 'Missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'Missing', 'Missing', 'Missing']],
      dtype=object)

In [63]:
x_train_clean

array([[65.0, 196.0, 2003.0, ..., 'Missing', 'Missing', 'Missing'],
       [80.0, 0.0, 1976.0, ..., 'Missing', 'Missing', 'Missing'],
       [68.0, 162.0, 2001.0, ..., 'Missing', 'Missing', 'Missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'Missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'Missing', 'Missing', 'Missing'],
       [75.0, 0.0, 1965.0, ..., 'Missing', 'Missing', 'Missing']],
      dtype=object)

In [64]:
x_train_clean_var = pd.DataFrame(x_train_clean , columns = num_var_mean + num_var_median + cat_var_mode + cat_var_missing)
x_test_clean_var = pd.DataFrame(x_test_clean , columns = num_var_mean + num_var_median + cat_var_mode + cat_var_missing)


In [65]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [66]:
x_train_clean_var

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65,196,2003,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,Missing,Missing
1,80,0,1976,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
2,68,162,2001,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
3,60,0,1998,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,Missing,Missing,Missing
4,84,350,2000,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
5,85,0,1993,Grvl,None,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,Unf,TA,TA,Missing,MnPrv,Shed
6,75,186,2004,Grvl,Stone,Ex,TA,Av,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,Missing,Missing
7,70.05,240,1973,Grvl,Stone,Gd,TA,Mn,ALQ,BLQ,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Shed
8,51,0,1931,Grvl,None,TA,TA,No,Unf,Unf,FuseF,TA,Detchd,Unf,Fa,TA,Missing,Missing,Missing
9,50,0,1939,Grvl,None,TA,TA,No,GLQ,Unf,SBrkr,TA,Attchd,RFn,Gd,TA,Missing,Missing,Missing


In [67]:
x_train_clean_var.isnull().mean()

LotFrontage     0.0
MasVnrArea      0.0
GarageYrBlt     0.0
Alley           0.0
MasVnrType      0.0
BsmtQual        0.0
BsmtCond        0.0
BsmtExposure    0.0
BsmtFinType1    0.0
BsmtFinType2    0.0
Electrical      0.0
FireplaceQu     0.0
GarageType      0.0
GarageFinish    0.0
GarageQual      0.0
GarageCond      0.0
PoolQC          0.0
Fence           0.0
MiscFeature     0.0
dtype: float64

In [68]:
x_test_clean_var.isnull().mean()

LotFrontage     0.0
MasVnrArea      0.0
GarageYrBlt     0.0
Alley           0.0
MasVnrType      0.0
BsmtQual        0.0
BsmtCond        0.0
BsmtExposure    0.0
BsmtFinType1    0.0
BsmtFinType2    0.0
Electrical      0.0
FireplaceQu     0.0
GarageType      0.0
GarageFinish    0.0
GarageQual      0.0
GarageCond      0.0
PoolQC          0.0
Fence           0.0
MiscFeature     0.0
dtype: float64

In [69]:
train['GarageFinish'].value_counts()

Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64

In [70]:
x_test_clean_var['GarageFinish'].value_counts()

Unf        625
RFn        389
Fin        367
Missing     78
Name: GarageFinish, dtype: int64

In [71]:
x_test_clean_var['Alley'].value_counts()

Grvl    1422
Pave      37
Name: Alley, dtype: int64

In [72]:
percentages

Id                0.000000
MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      17.739726
LotArea           0.000000
Street            0.000000
Alley            93.767123
LotShape          0.000000
LandContour       0.000000
Utilities         0.000000
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.000000
Exterior2nd       0.000000
MasVnrType        0.547945
MasVnrArea        0.547945
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
BsmtQual          2.534247
BsmtCond          2.534247
BsmtExposure      2.602740
BsmtFinType1      2.534247
BsmtFinSF1        0.000000
BsmtFinType2      2.602740
BsmtFinSF2        0.000000
B

# Home Work (Adding all columns in one dataset)

In [73]:
remainder_train_index = [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   61,
   62,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   75,
   76,
   77,
   78,
   79]

In [76]:
remaining_vars = [percentages.keys()[index] for index in remainder_train_index]
remaining_vars

['Id',
 'MSSubClass',
 'MSZoning',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

In [77]:
len(remaining_vars)

61

In [79]:
train = pd.concat( [x_train_clean_var , train[remaining_vars]] ,axis=1 )
train.shape

(1460, 80)

In [80]:
train.isnull().mean()*100

LotFrontage      0.0
MasVnrArea       0.0
GarageYrBlt      0.0
Alley            0.0
MasVnrType       0.0
BsmtQual         0.0
BsmtCond         0.0
BsmtExposure     0.0
BsmtFinType1     0.0
BsmtFinType2     0.0
Electrical       0.0
FireplaceQu      0.0
GarageType       0.0
GarageFinish     0.0
GarageQual       0.0
GarageCond       0.0
PoolQC           0.0
Fence            0.0
MiscFeature      0.0
Id               0.0
MSSubClass       0.0
MSZoning         0.0
LotArea          0.0
Street           0.0
LotShape         0.0
LandContour      0.0
Utilities        0.0
LotConfig        0.0
LandSlope        0.0
Neighborhood     0.0
Condition1       0.0
Condition2       0.0
BldgType         0.0
HouseStyle       0.0
OverallQual      0.0
OverallCond      0.0
YearBuilt        0.0
YearRemodAdd     0.0
RoofStyle        0.0
RoofMatl         0.0
Exterior1st      0.0
Exterior2nd      0.0
ExterQual        0.0
ExterCond        0.0
Foundation       0.0
BsmtFinSF1       0.0
BsmtFinSF2       0.0
BsmtUnfSF    

In [82]:
x_train_clean_var.shape

(1460, 19)